In [9]:
import pandas as pd

from collections import Counter

# utils
import os
import re
from tqdm import tqdm
tqdm.pandas()
from collections import Counter

# text processing
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
# nltk.download('stopwords')
# nltk.download('wordnet')
stopwords = set(stopwords.words('english'))

train_path = 'imdb/plain_text/train-00000-of-00001.parquet'
test_path = 'imdb/plain_text/test-00000-of-00001.parquet'

In [10]:
df_train = pd.read_parquet(train_path)
df_test = pd.read_parquet(test_path)

df = pd.concat([df_train, df_test], ignore_index=True, sort=False)

# print(df.shape)
# print(df.head()) 

In [11]:
def rm_link(text):
  return re.sub(r'https?://\S+|www\.\S+', '', text)

def rm_html(text):
  return re.sub(r'<[^>]+>', '', text)

def space_bt_punct(text):
  pattern = r'([.,!?-])'
  s = re.sub(pattern, r' \1 ', text)     # add whitespaces between punctuation
  s = re.sub(r'\s{2,}', ' ', s)        # remove double whitespaces    
  return s

def clean_html(text):
  # Remove specific sequences
  text = re.sub(r"<br|/><br|/>", "", text)
  return text

def rm_punct(text):
  return re.sub(r'[\"\#\$\%\&\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~\.\!\?\-\,]', ' ', text)

df['text'] = df['text'].apply(rm_link)
df['text'] = df['text'].apply(rm_html)
df['text'] = df['text'].apply(space_bt_punct)
df['text'] = df['text'].apply(clean_html)
df['text'] = df['text'].apply(rm_punct)

df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

df['text'] = df['text'].apply(lambda x: x.lower())

In [12]:
# from collections import Counter
# VOCABULARY_SIZE = 20000

# # Assuming train_data is a list of sentences
# def build_vocab(train_data, max_size):
#   counter = Counter(word for sentence in train_data for word in sentence.split())
#   most_common = counter.most_common(max_size)
#   vocab = {word: i for i, (word, _) in enumerate(most_common, start=2)}
#   vocab['<pad>'] = 0
#   vocab['<unk>'] = 1
#   return vocab, most_common

# # Create vocab from train_data
# train_data = df['text'].tolist()
# TEXT_vocab, most_common_words = build_vocab(train_data, VOCABULARY_SIZE)
# print(f'Vocabulary size: {len(TEXT_vocab)}')

# # print("Most common words with their frequencies:")
# # for word, freq in most_common_words:
# #   print(f"{word}: {freq}")

# # Convert the 'text' column to a list
# text_list = df['text'].tolist()

# # Open a file in write mode
# with open('output.txt', 'w') as file:
#   # Write each item on a new line
#   for line in text_list:
#     file.write(f"{line}\n")

In [13]:
# df['token_length'] = df.text.progress_apply(lambda x: len(x.split()))
# # df.head()
# # df['token_length'].describe()

In [14]:
# preprocessing
def tokenize(text):
  return word_tokenize(text)

def rm_stopwords(text):
  return [i for i in text if i not in stopwords]

def lemmatize(text):
  lemmatizer = WordNetLemmatizer()    
  lemmas = [lemmatizer.lemmatize(t) for t in text]
  # make sure lemmas does not contains sotpwords
  return rm_stopwords(lemmas)

def preprocess_pipeline(text):
  tokens = tokenize(text)
  no_stopwords = rm_stopwords(tokens)
  lemmas = lemmatize(no_stopwords)
  return ' '.join(lemmas)

In [15]:
df.head()

,text,label
0,i rented i am curious yellow video store contr...,0
1,i am curious yellow risible pretentious steami...,0
2,if avoid making type film future this film int...,0
3,this film probably inspired godard masculin fé...,0
4,oh brother hearing ridiculous film umpteen yea...,0


In [17]:
df.tail()

,text,label
49995,just got around seeing monster man yesterday i...,1
49996,i got part competition prize i watched really ...,1
49997,i got monster man box set three films i mainly...,1
49998,five minutes started feel naff looking got com...,1
49999,i caught movie sci fi channel recently it actu...,1


In [16]:
df[['text', 'label']].to_csv('./imdb_processed.csv', index=False, header=True)